<a href="https://colab.research.google.com/github/jeosol/langchain-study/blob/main/samw-yt/YT_Chatting_with_Flan20B_UL2_using_LangChain_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install huggingface_hub langchain transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00


In [2]:
!pip show langchain

Name: langchain
Version: 0.0.310
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [4]:
from langchain.llms import HuggingFaceHub

In [5]:
flan_ul2 = HuggingFaceHub(
    repo_id="google/flan-ul2",
    model_kwargs={"temperature":0.1,
                  "max_new_tokens":256})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":0 }
)

In [7]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [8]:
memory = ConversationBufferMemory()

In [9]:
conversation = ConversationChain(
    llm=flan_ul2,
    verbose=True,
    memory=memory
)

In [10]:
conversation.predict(input="Hi there! I am Sam")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Sam
AI:

> Finished chain.


'Hi Sam, how can I help you?'

In [ ]:
 conversation.predict(input="How are you today?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Sam
AI: Hi Sam, how can I help you?
Human: How are you today?
AI:

> Finished chain.


'I am good, thanks.'

In [ ]:
conversation.predict(input=" Can you help me with some customer support?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Sam
AI: Hi Sam, how can I help you?
Human: How are you today?
AI: I am good, thanks.
Human:  Can you help me with some customer support?
AI:

> Finished chain.


'I can help you with that.'

In [ ]:
conversation.predict(input="My TV is broken. can you help fix it?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Sam
AI: Hi Sam, how can I help you?
Human: How are you today?
AI: I am good, thanks.
Human:  Can you help me with some customer support?
AI: I can help you with that.
Human: My TV is broken. can you help fix it?
AI:

> Finished chain.


'I can help you with that.'

## Counting the tokens

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

In [ ]:
test_input = "Now Good Morning Ms Rogers"
# tokenizer([test_input])
tokenizer.tokenize(test_input)

['▁Now', '▁Good', '▁Morning', '▁M', 's', '▁Roger', 's']

In [ ]:
conversation.memory.buffer

'\nHuman: Hi there! I am Sam\nAI: Hi Sam, how can I help you?\nHuman: How are you today?\nAI: I am good, thanks.\nHuman:  Can you help me with some customer support?\nAI: I can help you with that.\nHuman: My TV is broken. can you help fix it?\nAI: I can help you with that.'

In [ ]:
formatted_prompt = conversation.prompt.format(input='the next input',history=memory.buffer)
formatted_prompt

'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n\nHuman: Hi there! I am Sam\nAI: Hi Sam, how can I help you?\nHuman: How are you today?\nAI: I am good, thanks.\nHuman:  Can you help me with some customer support?\nAI: I can help you with that.\nHuman: My TV is broken. can you help fix it?\nAI: I can help you with that.\nHuman: the next input\nAI:'

In [ ]:
def chat_to_llm(chat_llm ):
    conversation_total_tokens = 0
    new_conversation = ConversationChain(llm=chat_llm,
                                     verbose=False,
                                     memory=ConversationBufferMemory()
                                     )

    while True:
        message = input("Human: ")
        if message=='exit':
            print(f"{conversation_total_tokens} tokens used in total in this conversation")
            break
        if message:
            formatted_prompt = conversation.prompt.format(input=message,history=new_conversation.memory.buffer)
            num_tokens = len(tokenizer.tokenize(formatted_prompt))
            conversation_total_tokens += num_tokens
            print(f'tokens sent {num_tokens}')
            response = new_conversation.predict(input=message)
            response_num_tokens = len(tokenizer.tokenize(response))
            conversation_total_tokens += response_num_tokens
            print(f"LLM: {response}")




In [ ]:
chat_to_llm(flan_ul2)

Human: Hi how are you today?
tokens sent 65
LLM: I am fine. How can I help you?
Human: Are you male or female?
tokens sent 85
LLM: I am a female.
Human: are you tall or short?
tokens sent 101
LLM: I am tall.
Human: How tall are you?
tokens sent 114
LLM: I am 5'9" tall.
Human: exit
393 tokens used in total in this conversation


In [ ]:
chat_to_llm(flan_t5)


Human: hi how are you?
tokens sent 64
LLM: I am good thanks.
Human: What is your name?
tokens sent 78
LLM: I am a robot.
Human: exit
153 tokens used in total in this conversation


## Using a Summary Memory

In [ ]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

In [ ]:
summary_memory = ConversationSummaryMemory(llm=flan_ul2)

conversation = ConversationChain(
    llm=flan_ul2,
    verbose=True,
    memory=summary_memory
)

In [ ]:
conversation.predict(input="Hi there! I am Sam")

In [ ]:
conversation.predict(input="How are you today?")

In [ ]:
conversation.predict(input="I am good. What is your name?")

In [ ]:
conversation.predict(input="Thats cool can you help me with customer support?")

In [ ]:
conversation.predict(input="What time is your shop open today?")